# SISU Digital Humanities: Textual and Language Analysis on Social Media<br />
### Session 3: Topic modeling
Created by Tom van Nuenen (tom.van_nuenen@kcl.ac.uk) <br />


# Topic modeling

This notebook introduces topic modeling. Topic modeling is a type of statistical modeling for the discovery of abstract "topics" that occur in a collection of documents. It is frequently used in NLP to aid the discovery of hidden semantic structures in a collection of texts.

We'll use the `Gensim` package to create our topic models, as it allows us to run tests to optimize our topic amount.

After reading this notebook, you'll be able to:

1. Use gensim to create topic models;
2. Explore the topic models using PyLDAvis;
3. Evaluate topic models using several methods.

In [2]:
import logging
import string
import os
import pickle
from pprint import pprint
import re 
import numpy as np
import pandas as pd
from IPython.display import clear_output
from more_itertools import chunked

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Gensim
from gensim import corpora, models, similarities
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import gensim

# SpaCy 
import spacy
!spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

# Logging if you want it
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Suppressing it should you want to
logging.getLogger().setLevel(logging.CRITICAL)

# Suppressing warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

#SKLearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomvannuenen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


/Users/tomvannuenen/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## Getting the data

We'll use the same two datasets as we did in a previous notebook: comments and submissions from The Red Pill, an infamous US-based internet community on men's rights.

*Trigger warning: this dataset includes a lot of misogynist language. We are expecting to find topics that reflect the problematic politics of this community.*

In [ ]:
trp_sub = pd.read_csv("data/TRP-submissions.csv", lineterminator='\n')
trp_com = pd.read_csv("data/TRP-comments.csv", lineterminator='\n')

In [ ]:
# OPTIONAL - use SISU MOOC data
# df = pd.read_csv("data/icc3-comments.csv", lineterminator='\n') 

Get rid of the empty cells in our two DataFrames using the `trp_sub[~trp_sub['selftext'].isin(['[removed]', '[deleted]' ])]` and `.dropna(subset=['selftext'])` (for the submissions DF) and `.dropna(subset=['body'])` (for the comments DF).

In [ ]:
# Your code here

trp_sub = trp_sub[~trp_sub['selftext'].isin(['[removed]', '[deleted]' ])].dropna(subset=['selftext'])
trp_com = trp_com[~trp_com['body'].isin(['[removed]', '[deleted]' ])].dropna(subset=['body'])

## Concatenating submissions and comments

For the topic model we're going to make, we will first concatenate the associated submissions and comments (i.e., the threads).

Using the `pd.merge()` method, we can do an "inner join" of our two DataFrames. This is a relational database operation, which is a common operation in SQL. See [here](https://www.w3schools.com/sql/sql_join.asp) if you want to learn more. An "inner join" will yield a new DF which only contains those submissions that have associated comments (based on their "idstr" and "parent" values).

![alt text](https://s33046.pcdn.co/wp-content/uploads/2019/06/venn-diagram-representation-of-sql-inner-join-.png)

In [ ]:
# merge DF based on idstr and parent
trp_t = pd.merge(trp_sub, trp_com, how='inner', left_on='idstr', right_on='parent')

We now have a DataFrame containing all original posts / submissions, *and* the comments associated with this original post! Note that this yields a lot of rows with the same "selftext" value (as lots of comments refer to the same original post).

Also note that this operations has changed the names of the columns: we now have "x" and "y" columns based on the original DataFrame we took the data from. "idstr_x", for instance, refers to the "idstr" column from the `trp_sub` DataFrame.

In [ ]:
trp_t.head()

Now, let's iterate over our merged DataFrame and group all associated submissions and comments together. 
- We first create an empty dictionary.
- We then iterate over our DataFrame using the Pandas `.iterrows()` method, which allows us to iterate over rows in a DataFrame in a for-loop. It yields a tuple consisting of the index and the row.
- `If` the "idstr_x" column does not yet exist in the keys of our new dictionary, add it. As the value, add the "selftext" (i.e., the original post) and "body" (i.e., the first comment) columns.
- `Else`, if we have the key already, only add the "body" column (i.e., the comment). 

In [ ]:
data_d = {}
for i, r in trp_t.iterrows():
    if r.idstr_x not in data_d.keys():
        data_d[r.idstr_x] = [r.selftext, r.body]    
    else:
        data_d[r.idstr_x].append(r.body)    

In [ ]:
# See if it works
data_d['t3_3n0dg0']

Finally, we'll join the items in each of the values in our `dict` and put that in a list:

In [ ]:
data = ['\n'.join(thread) for thread in data_d.values()]

## Preprocessing

First, let's get rid of the newlines. Iterate over each text in our `data` list, and use string replacements or RegEx to remove the newlines.

In [4]:
# Your code here

data = [re.sub(r'\s+', ' ', txt) for txt in data]
data = [txt.replace('\n', ' ') for txt in data]

#data = [re.sub(r'\s+', ' ', txt) for txt in df.text]
#data = [txt.replace('\n', ' ') for txt in df.text]


Time to tokenize. Let's use Gensim's `simple_preprocess()` method this time. If you haven't seen `yield` before, it is used in what's called a generator function. This is simply a function that iterates, instead of only returning something once.

`Return` sends a specified value back to its caller whereas `yield` can produce a sequence of values. We should use `yield` when we want to iterate over a sequence, but don’t want to store the entire sequence in memory.

In [6]:
def tokenizer(texts):
    for text in texts:
        yield(gensim.utils.simple_preprocess(text, deacc=True))

tokens_list = list(tokenizer(data))

How many threads do we have?

In [7]:
len(tokens_list)

16438

In [8]:
tokens_list[1][:10]

['our',
 'educator',
 'and',
 'mentor',
 'team',
 'is',
 'eager',
 'to',
 'know',
 'what']

And how many tokens in total?

In [9]:
counter = 0
for each in tokens_list:
    counter += len(each)
counter

940882

In [10]:
tokens_list[1][:10]

['our',
 'educator',
 'and',
 'mentor',
 'team',
 'is',
 'eager',
 'to',
 'know',
 'what']

### Creating N-grams with Gensim

Topic modeling – as well as many other kinds of NLP methods – works better when using N-grams, as this allows words that frequently appearing together to be concatenated (e.g. "red pill" means something different than "red" and "pill" separately).

Gensim’s `Phrases` model implements bigrams, trigrams, quadgrams, etc. 

`Phrases` detects phrases based on collocation counts. It builds a model of input text that you then can use on other data.

Gensim detects a bigram if a scoring function for two words exceeds a threshold. The two important arguments to `Phrases` are `min_count` and `threshold`. The higher the values of these parameters, the harder it is for words to be combined to bigrams.

In [11]:
# Build bigram and trigram models
# min_count = min amount of tokens/bigrams in corpus. Threshold = score-based; higher = fewer phrases.
bigram = gensim.models.Phrases(tokens_list, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[tokens_list], threshold=100)  

# `Phraser` must be built from an initial `Phrases` instance. 
# It then works faster while using much less memory. See https://radimrehurek.com/gensim/models/phrases.html
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Stopwords removal, bigrams, lemmatization 
Let's define some functions for stopword removal, making bigrams and trigrams, and lemmatization (we will use SpaCy for the latter - see [here](https://spacy.io/api/annotation) for more info).


In [12]:
# prepare stopwords
stop = set(stopwords.words('english') + ['’', '“', '”', 'nbsp', 'http'])

In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop] for doc in texts]
      
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for doc in texts:
        joined = nlp(" ".join(doc)) 
        texts_out.append([token.lemma_ for token in joined if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove stopwords
tokens_no_stops = remove_stopwords(tokens_list)

# Form trigrams
trigrams = make_trigrams(tokens_no_stops)

# Do lemmatization 
lemmas = lemmatization(trigrams)


In [15]:
lemmas[1][:10]

['educator', 'hope', 'course', 'hope', 'goal']

### Pickling
We've done a lot of work, so let's save this object in a pickle. Any object in Python can be pickled so that it can be saved on disk (or in this case, a virtual disk). What pickle does is that it “serializes” the object first before writing it to file. Pickling is a way to convert a python object (list, dict, etc.) into a character stream. The idea is that this character stream contains all the information necessary to reconstruct the object in another python script – or to fetch it at a later time when you don't want to do the computing work all over again.

Note that when pickling, you can use any file extension name you want.

In [ ]:
# Save to disk
with open("TRP_nouns.text", "wb") as docP: 
    pickle.dump(lemmas, docP)

In [ ]:
!ls

In [ ]:
# should you want to load in the pickle we just saved:
with open("TRP_nouns.text", "rb") as docP: 
    lemmas = pickle.load(docP)

## Creating a `Dictionary` with Gensim

Now, let's create our gensim dictionary - a mapping of each word to a unique id. 
It will be used to create a `Corpus` object, which is gensim’s equivalent of a Document-Term matrix.

In [16]:
# Create Dictionary 
dictionary = corpora.Dictionary(lemmas)

# Create Corpus, i.e. Document-Term Matrix
corpus = [dictionary.doc2bow(text) for text in lemmas]

In [ ]:
dictionary

*Note that Gensim allows you to run the `.add_documents` method that will append additional documents to your `dictionary`, if you need to.*

Let's view some of the corpus we have now:

In [ ]:
print(corpus[0][:10])

Observe the first 10 tuples above. Each consists of words with a unique id. This a mapping of (word_id, word_frequency). For example, (0, 1) above demonstrates that word id 0 occurs once in the first document. Word id 5 occurs 4 times, and so on. This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [ ]:
dictionary[5]

And if you want to see the associated id for some word:

In [ ]:
dictionary.token2id['advance']

## Running an LDA model

Now, let's run a Gensim LDA model:

In [21]:
## Build LDA model. Make sure to play around with chunksize and passes and check if coherence score changes a lot.

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=5, 
                                           random_state=100,
                                           # eval_every = 20, # this is evaluation, perplexity
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## Visualizing the model

Let's try to evaluate our topics. First, we can visualize our topics using pyLDAvis. A "good" topic model produces non-overlapping, fairly large bubbles, which should be scattered throughout the chart instead of being clustered in one quadrant. A model with too many topics will typically have many overlaps, small sized bubbles clustered in one region of the chart. **This is the first way in which you can evaluate your topic models**.

In [22]:
# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.295098  0.266153       1        1  36.738736
3     -0.206663 -0.327921       2        1  32.486440
2      0.177647  0.048124       3        1  12.269898
1      0.151471  0.006259       4        1   9.749756
0      0.172643  0.007385       5        1   8.755170, topic_info=            Term         Freq        Total Category  logprob  loglift
11       culture  6258.000000  6258.000000  Default  30.0000  30.0000
60        people  5980.000000  5980.000000  Default  29.0000  29.0000
17          time  4264.000000  4264.000000  Default  28.0000  28.0000
2         course  1520.000000  1520.000000  Default  27.0000  27.0000
97        friend  1308.000000  1308.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
5        learner   108.537915   109.308485   Topic5  -4.9534   2.4285
95      decision   109.127392   109.908756   Topic5  -4.9480   2.4284
1246    feedback    95.111067    95.882557   Topic5  -5.0854   2.4274
656   definition    92.257056    93.027451   Topic5  -5.1159   2.4272
1273        poem    90.088198    90.861256   Topic5  -5.1397   2.4270

[212 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
568       1  0.006702    activity
568       2  0.991908    activity
1587      3  0.993413  adjustment
301       1  0.033827         age
301       2  0.007517         age
...     ...       ...         ...
90        2  0.798195        work
175       1  0.012558       world
175       2  0.972679       world
175       3  0.013700       world
75        2  0.998679        year

[224 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 2, 1])

### PyLDAvis graph

On the left, there is a 2D plot of the "distance" between all of the topics (labeled as the Intertopic Distance Map). This plot uses a multidimensional scaling (MDS) algorithm. 
- Similar topics should appear close together on the plot; dissimilar topics should appear far apart. 
- The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.

### Exploring topics and words
- You can scrutinize a topic more closely by clicking on its circle, or entering its number in the "selected topic" box in the upper-left (Note that, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics.)
- If you roll your mouse over a term in the bar chart on the right, the topic circles will resize in the plot on the left. This shows the strength of the relationship between the topics and the selected term.

### Salience
On the right, there is a bar chart with the top terms. When no topic is selected in the plot on the left, the bar chart shows the top-30 most **salient** terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.

### Probability Vs Exclusivity 
When you select a particular topic, this bar chart changes to show the top-30 most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter λ, which can be adjusted with a slider above the bar chart:

* Setting λ close to 1.0 (the default) will rank the terms according to their probability within the topic.
* Setting λ close to 0.0 will rank the terms according to their "distinctiveness" or "exclusivity" within the topic. This means that terms that occur only in this topic, and do not occur in other topics.

You can move the slider between 0.0 and 1.0 to weigh term probability and exclusivity.

## Assignment: Analyzing the pyLDA visualization

The interactive visualization pyLDAvis produces is helpful for both **individual** topics: you can manually select each topic to view its top most frequent and/or "relevant" terms, using different values of the λ parameter. This can help when you're trying to assign a name or "meaning" to each topic. 

It also helps you to see the **relationships** between topics: exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

See if you can make sense of the patterns you are seeing. Can you give an explanation for the distance between certain topics?

In [ ]:
# ADDING A DOC
test_doc = ['alpha men are better than beta men.', 'indeed, alpha men are much cooler.']
bow_test_doc = dictionary.doc2bow(test_doc)
lda_model.get_document_topics(bow_test_doc)


In [ ]:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

## Calculating Topic Coherence and Perplexity

Next, we can apply some statistical measures to help us determine the optimal number of topics in our topic model.

**Topic Coherence** is a measure applied to the top N words from each topic. It is defined as the average / median of the pairwise word-similarity scores of the words in the topic. This helps to distinguish between topics that are semantically interpretable topics, and topics that are artifacts of statistical inference. 

A set of statements or facts is said to be coherent if the statements support each other. An example of a coherent fact set is “the game is a team sport”, “the game is played with a ball”, “the game demands great physical efforts”

A good model will generate topics with *high* topic coherence scores. Good topics are topics that can be described by a short label, therefore this is what the topic coherence measure should capture.

There are different ways to calculate semantic similarity, e.g. through normalized pointwise mutual information (NPMI) or through cosine similarity.

**Perplexity** is a measure of how well a probability model predicts a sample.
It captures how surprised a model is of new data it has not seen before, and is measured as the normalized log-likelihood of a held-out test set. The model with the *lowest* perplexity is generally considered the “best”. 

*The issue with perplexity is that it tends to not be strongly correlated to human judgment and, even sometimes slightly anti-correlated. Therefore, we'll work with coherence scores*.

In [ ]:
%%time
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # A measure of how good the model is. The lower the better.

# Compute Coherence Score
coherence_model = CoherenceModel(model=lda_model, corpus=corpus, texts=lemmas, dictionary=dictionary, coherence='c_v') 
# The higher the better. A coherence score of .4 means probably not right number of topics; .6 is great. Anything more is suspiciously great.
coherence = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence)

There's no hard or fast rule on what makes a good coherence or perplexity score. We have to compare this for different iterations of our topic model (using different amounts of topics) to see which one works best.

## Optimizing coherence scores

The most obvious thing we can do to find optimal scores is to play around with the amount of topics our model creates. One way to do this is to build many LDA models with different values of number of topics (k), and then pick the one that gives the highest coherence value. Choosing a ‘k’ at the end of a rapid growth of topic coherence usually yields meaningful and interpretable topics. If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

This `compute_coherence_values()` function trains multiple LDA models, provides the models, and tells you their corresponding coherence scores.

Also note the docstring I create here: these are documentation for the functions we create. It describes what a function does, and can be called using `help(function_X)`.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    total_amount = limit / step
    current_amount = 0
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100, update_every=1, 
                                chunksize=500, passes=10, alpha='auto', per_word_topics=False)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        # When using 'c_v' texts should be provided, corpus isn’t needed. 
        # When using ‘u_mass’ corpus should be provided, if texts is provided, it will be converted to corpus using the dictionary 
        coherence_values.append(coherencemodel.get_coherence())
        current_amount += 1
        print("Built " + str(current_amount) + " of " + str(total_amount) + " models")

    return model_list, coherence_values


In [ ]:
help(compute_coherence_values)

Using our new function, let's run a bunch of topic models with different amounts of topics.

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=lemmas, start=10, limit=100, step=10)

Now, from all those models, let's visualize the output of the coherence scores.

In [ ]:
# Show graph
limit=100; start=10; step=10;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print these coherence scores
for m, cv in zip(x, coherence_values):
    print(" Num Topics =", m, "Coherence Value =", round(cv, 4))

If the coherence score seems to keep increasing, it generally makes sense to pick the model that gave the highest CV before dropping again. We'll pick the most fitting amount of topics to continue our research.

In [ ]:
# Save to disk
with open("optimal_model.model", "wb") as docP: 
    pickle.dump(model_list[4], docP)

In [ ]:
!ls

In [ ]:
# should you want to load in the pickle we just saved:
with open("optimal_model.model", "rb") as docP: 
    optimal_model = pickle.load(docP)

In [ ]:
# Select the ideal model and print the topics
optimal_model = model_list[4]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=20))

## Finding most dominant topic per thread

One of the practical application of topic modeling is to determine what topic a given Reddit thread is about. To figure this out, we find the topic number that has the highest percentage contribution in that thread. We'll write a `dominant_topic()` function that aggregates this information in a presentable table.

In [ ]:
optimal_model[corpus]

In [ ]:
def dominant_topic(ldamodel=optimal_model, corpus=corpus, texts=lemmas):
    # Create DF
    thread_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each thread
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                thread_topics_df = thread_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    thread_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    thread_topics_df = pd.concat([thread_topics_df, contents], axis=1)
    return thread_topics_df 

df_topic_thread_keywords = dominant_topic(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_thread_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic

Retrieving dominant topics via `.loc`

In [ ]:
df_dominant_topic.loc[df_dominant_topic['Dominant_Topic'] == 4.0]

## Finding most distinctive threads per topic

We can also find the threads that include the highest amount of words for a certain topic. You could use this if you have found a really interesting topic, and you want to know the top threads this topic is typically found in. 

In [ ]:
# Group top 5 threads under each topic
thread_topics_sorteddf = pd.DataFrame()

thread_topics_outdf_grpd = df_topic_thread_keywords.groupby('Dominant_Topic')

for i, grp in thread_topics_outdf_grpd:
    thread_topics_sorteddf = pd.concat([thread_topics_sorteddf, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(5)], 
                                            axis=0)

# Reset Index    
thread_topics_sorteddf.reset_index(drop=True, inplace=True)

# Format
thread_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
thread_topics_sorteddf

In [ ]:
thread_topics_sorteddf['Text'][4]

## Topic distribution across threads

Finally, we can look at the volume and distribution of topics in order to judge how widely each topic is discussed in our threads. The below table exposes that information.

In [ ]:
# Number of Threads for Each Topic
topic_counts = df_topic_thread_keywords['Dominant_Topic'].value_counts()

# Percentage of Threads for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_thread_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics[:10]